In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


In [3]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))


def sigmoid_derivative(z):
    return z * (1.0 - z)

### Node Class

In [4]:
class Neuron:
    def __init__(self, weights=None, delta=0.0, output=0.0):
        if weights is None:
            weights = []
        self.weights = weights
        self.delta = delta
        self.output = output

    def set_delta(self, error):
        self.delta = error * sigmoid_derivative(self.output)
    
    def weighted_sum(self, inputs):
        ws = 0.0
        for i in range(len(self.weights)):
            ws += self.weights[i] * inputs[i]
        return ws
    
    def activate(self, inputs):
        output = sigmoid(self.weighted_sum(inputs))
        self.output = output
        return output

### Layer Class

In [13]:
class Layer:
    def __init__(self, neurons, bias_neuron=None):
        if bias_neuron is None:
            bias_neuron = Neuron()
        self.neurons = neurons
        self.bias_neuron = bias_neuron

    @property
    def all_outputs(self):
        total_neurons = [self.bias_neuron] + self.neurons  
        return [neuron.output for neuron in total_neurons]


    
    def activate_neurons(self, inputs):
        # Activate all neurons in the layer with given inputs
        return [neuron.activate(inputs) for neuron in self.neurons]
    
    def total_delta(self, previous_layer_neuron_idx):
        # total  for a given neuron index in the previous layer
        total = self.neurons
        return sum(
            neuron.weights[previous_layer_neuron_idx] * neuron.delta
            for neuron in total
        )


### Network Class

In [14]:
class Network:
    def __init__(self, hidden_layers, output_layer, learning_rate):
        self.hidden_layers = hidden_layers
        self.output_layer = output_layer
        self.learning_rate = learning_rate

    def layers(self):
        return self.hidden_layers + [self.output_layer]

    def feed_forward(self, inputs):
        for layer in self.hidden_layers:
            inputs = [1.0,] + layer.activate_neurons(inputs)
        return self.output_layer.activate_neurons(inputs)

    def derivative_error_to_output(self, actual, expected):
        return [actual[i] - expected[i] for i in range(len(actual))]

    def back_propagate(self, inputs, errors):
        for index, neuron in enumerate(self.output_layer.neurons):
            neuron.set_delta(errors[index])

        for layer_idx in reversed(range(len(self.hidden_layers))):
            layer = self.hidden_layers[layer_idx]
            next_layer = (
                self.output_layer
                if layer_idx == len(self.hidden_layers) - 1
                else self.hidden_layers[layer_idx + 1]
            )
            for neuron_idx, neuron in enumerate(layer.neurons):
                error_from_next_layer = next_layer.total_delta(neuron_idx)
                neuron.set_delta(error_from_next_layer)

        self.update_weights_for_all_layers(inputs)

    def update_weights_for_all_layers(self, inputs):
        for layer_idx in range(len(self.hidden_layers)):
            layer = self.hidden_layers[layer_idx]
            previous_layer_outputs = (
                inputs
                if layer_idx == 0
                else self.hidden_layers[layer_idx - 1].all_outputs
            )
            for neuron in layer.neurons:
                self.update_weights_in_a_layer(previous_layer_outputs, neuron)

        for neuron in self.output_layer.neurons:
            self.update_weights_in_a_layer(self.hidden_layers[-1].all_outputs, neuron)

    def update_weights_in_a_layer(self, previous_layer_outputs, neuron):
        for idx in range(len(previous_layer_outputs)):
            neuron.weights[idx] -= (
                self.learning_rate * neuron.delta * previous_layer_outputs[idx]
            )

    def train(self, num_epoch, num_outputs, training_set, training_output):
        for epoch in range(num_epoch):
            sum_error = 0.0
            for idx, row in enumerate(training_set):
                expected = [0 for _ in range(num_outputs)]
                expected[training_output[idx]] = 1 
                actual = self.feed_forward(row)
                errors = self.derivative_error_to_output(actual, expected)
                
                self.back_propagate(row, errors)
                sum_error += self.mse(actual, expected)
            print(f"Mean squared error: {sum_error}")
            print(f"epoch={epoch}")

    def predict(self, inputs):
        outputs = self.feed_forward(inputs)
        return outputs.index(max(outputs))

    def mse(self, actual, expected):
        return sum((actual[i] - expected[i]) ** 2 for i in range(len(actual))) / len(actual)

In [15]:
import random

dataset = [[1.2, 0.9, 0.5]]
expected = [1]

n_outputs = 2

hidden_layers = [
    Layer(
        neurons=[
            Neuron(weights=[0.7, 1.2, 0.9]), 
            Neuron(weights=[0.6, 0.3, -1.2])
        ],
        bias_neuron=Neuron(output=1.0, weights=[0.8])
    )
]
output_layer = Layer(
    neurons=[
        Neuron(weights=[0.8, -1.3, 1.1]),
        Neuron(weights=[0.9, 1.5, -0.3])
    ]
)

network = Network(hidden_layers=hidden_layers, output_layer=output_layer, learning_rate=0.5)

network.train(500, n_outputs, dataset, expected)

prediction = network.predict([1.2, 0.9, 0.5])
print(f"The Predicted Class is {prediction}")



Mean squared error: 0.16641321562289807
epoch=0
Mean squared error: 0.14780246622949603
epoch=1
Mean squared error: 0.13080902526882196
epoch=2
Mean squared error: 0.11561202206734682
epoch=3
Mean squared error: 0.10224789296814739
epoch=4
Mean squared error: 0.09064145369882565
epoch=5
Mean squared error: 0.08064549349910959
epoch=6
Mean squared error: 0.07207719795829785
epoch=7
Mean squared error: 0.06474548083251087
epoch=8
Mean squared error: 0.05846829171900943
epoch=9
Mean squared error: 0.05308168395466943
epoch=10
Mean squared error: 0.04844320111250134
epoch=11
Mean squared error: 0.04443185742940751
epoch=12
Mean squared error: 0.040946358479664755
epoch=13
Mean squared error: 0.03790260556750692
epoch=14
Mean squared error: 0.035231076708805334
epoch=15
Mean squared error: 0.0328743829307698
epoch=16
Mean squared error: 0.03078512412989774
epoch=17
Mean squared error: 0.028924073682669818
epoch=18
Mean squared error: 0.027258674176896504
epoch=19
Mean squared error: 0.02576